# Práctica Spark, Apache Kafka, Spark SQL

Autor: Ignacio Arias Barra

En la presente práctica se va a llevar a cabo el análisis de un dataset mediante técnicas de procesamiento de datos específicas de la tecnología Spark. Concretamente, Spark SQL y Spark Streaming. La base de datos a utilizar, se llama DATASETMotoGPQatar.csv, la cual contiene información sobre tweets que se escribieron durante el gran premio de moto GP.

### Spark context

En primer lugar definimos el contexto de spark y spark streaming

In [2]:
# Spark
import findspark
spark_path = "/home/nacho/spark"
findspark.init(spark_path)
import re
import pyspark
from pyspark.sql import SparkSession
import shutil
import os
import signal
import sys
import subprocess
spark = (SparkSession.builder
    .master("local[*]")
    .config("spark.driver.cores", 1)
    .appName("understanding_sparksession")
    .getOrCreate() )
sc = spark.sparkContext

print(spark)
print(sc)

# Spark Streaming
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils
from operator import add
from operator import sub


### Spark SQL

#### Definición del esquema

Para la correcta lectura de la base de datos, necesitamos definir el esquema de la misma, aportando la tipología de las variables

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
customSchema = StructType([StructField("Id", LongType(), True),
                           StructField("Parent_sys_id", StringType(), True),
                           StructField("Source", StringType(), True),
                           StructField("Mentions", StringType(), True),
                           StructField("Target", StringType(), True),
                           StructField("Name_source", StringType(), True),
                           StructField("Body", StringType(), True),
                           StructField("Pub_date", TimestampType(), True),
                           StructField("URLs", StringType(), True),
                           StructField("Tipe_action", StringType(), True),
                           StructField("Link", StringType(), True),
                           StructField("Has_link", ByteType(), True),
                           StructField("Has_picture", ByteType(), True),
                           StructField("Website", StringType(), True),
                           StructField("Country", StringType(), True),
                           StructField("Activity", LongType(), True),
                           StructField("Followers", LongType(), True),
                           StructField("Following", LongType(), True),
                           StructField("Location", StringType(), True)
                          ])

#### Lectura de eventos

In [3]:
sep = '\t'
data_path = "data/DATASETMotoGP-Qatar.csv"
events = spark.read.csv(data_path, header=True, schema=customSchema, timestampFormat="dd/MM/yyyy HH:mm", sep = sep)

#### Visualización del esquema

In [4]:
events.printSchema()

root
 |-- Id: long (nullable = true)
 |-- Parent_sys_id: string (nullable = true)
 |-- Source: string (nullable = true)
 |-- Mentions: string (nullable = true)
 |-- Target: string (nullable = true)
 |-- Name_source: string (nullable = true)
 |-- Body: string (nullable = true)
 |-- Pub_date: timestamp (nullable = true)
 |-- URLs: string (nullable = true)
 |-- Tipe_action: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Has_link: byte (nullable = true)
 |-- Has_picture: byte (nullable = true)
 |-- Website: string (nullable = true)
 |-- Country: string (nullable = true)
 |-- Activity: long (nullable = true)
 |-- Followers: long (nullable = true)
 |-- Following: long (nullable = true)
 |-- Location: string (nullable = true)



#### Cuestiones

##### A) Contabilizar el número total de menciones a los pilotos Marc Márquez, Valentino Rossi y Dani Pedrosa.

In [5]:
Marc_id = 'marcmarquez93'

(events.select('Mentions')
.filter(events.Mentions.rlike(Marc_id))
.count())

58117

In [6]:
Valen_id = 'valeyellow46'

(events.select('Mentions')
.filter(events.Mentions.rlike(Valen_id))
.count())

61121

In [7]:
Dani_id = '26_danipedrosa'

(events.select('Mentions')
.filter(events.Mentions.rlike(Dani_id))
.count())

12342

##### B) Contabilizar los 5 países que más tweets han publicado (considerando los tweets que contengan dicha información).

In [8]:
countries_toshow = 5
(events.filter(events.Country != "not public")
 .groupBy("Country")
 .agg(count("Id").alias("tweets"))
 .orderBy("tweets", ascending=False)
 .limit(countries_toshow).show())

+-------+------+
|Country|tweets|
+-------+------+
|     es|172577|
|     us| 12722|
|     gb| 12588|
|     id|  8725|
|     it|  1843|
+-------+------+



##### C) Contabilizar los 3 hashtags más utilizados (que aparezcan el mayor número de veces) en el cuerpo de los tweets (campo "body").

In [9]:
hastags_toshow = 3
hastag_sym = '#'

(events.select('Body')
 .rdd
 .flatMap(lambda body: body.Body.split(' ') if body.Body is not None else 'empty')
 .map(lambda word: (word, 1) if word.startswith(hastag_sym) else (word, 0))
 .reduceByKey(lambda c1, c2: c1 + c2)
 .toDF()
 .select(col('_1').alias('Hastag'), col('_2').alias('Repetitions'))
 .orderBy('Repetitions', ascending=False)
 .limit(hastags_toshow)
 .show())

+-------+-----------+
| Hastag|Repetitions|
+-------+-----------+
|#motogp|      51961|
| #qatar|       9977|
| #moto3|       5797|
+-------+-----------+



### Spark Streaming + Apache Kafka

Para la ejecución de procesado streaming, ademáś de Spark, necesitamos de otras tecnologías como es el caso de Apache Kafka y Zookeeper.

Con la finalidad los métodos de instalación y uso de estas tecnologías, se ha optado por ejecutar un docker, en cuyo interior se ejecutarán tanto el broker kafka como zookeeper.

Este docker ha sido descargado de su repositorio en gitHub: *https://github.com/spotify/docker-kafka.git*

Una vez descargado, debemos ejecutar el siguiente comando en la terminal:

**sudo docker run -itd --name=kafka -p 2181:2181 -p 9092:9092 --env ADVERTISED_HOST=localhost --env ADVERTISED_PORT=9092 --env CONSUMER_THREADS=10 spotify/kafka**

Para llevar a cabo estos pasos de ejecución del docker, se presupone instalado el docker engine correspondiente.

El siguiente paso a tener en cuenta, es la activación de un producer. Éste se encargará de leer las líneas del fichero de la base de datos que contiene la información sobre los tweets y la enviará a kafka mediante el topic *Quatar_GP_2014*.

La forma de ejecutar el código es la siguiente:

**python kafka_producer.py 0.1 0.3 Quatar_GP_2014 data/DATASETMotoGP-Qatar.csv**

Antes de empezar a ejecutar el resto de la práctica, debemos definir el topic de lectura (mismo que el de escritura del producer) y la ip y puerto dónde se encuentre kafka (en nuestro caso en local, localhost)

In [3]:
topic = 'Quatar_GP_2014'
kafkaBrokerIPPort = 'localhost:9092'

#### Cuestiones

IMPORTANTE: En cada momento sólo podemos tener activo un sparkContext. El modo de ejecución de las cuestiones es el siguiente:

    1. Ejecutar cuadro con código de respuesta a la query
    2. Ejecutar el comando de activacióń del streaming, "scc.start"
    3. Ejecutar el comando de desactivación del streamin, "scc.stop(False)"

En caso de experimentar problemas, reiniciar el kernel del notebook y ejecutar de nuevo la query en cuestión.

##### A) Calcular el número total de menciones recibidas por cada cuenta de usuario durante el intervalo de 5 segundos.

In [11]:
# Context
time_interval = 5

sc = spark.sparkContext
ssc = StreamingContext(sc, time_interval)

kafkaParams = {"metadata.broker.list": kafkaBrokerIPPort}
stream = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams)

In [12]:
# ssc.checkpoint("checkpoint")

info_row = stream.map(lambda line: line[1].split('\t'))
mentions_name = info_row.flatMap(lambda mentions: mentions[3].split(','))
mention_map_names = mentions_name.map(lambda name: (name, 1) if name != '' else (name, 0))
mention_count = mention_map_names.transform(lambda rdd: rdd.filter(lambda x: x[1] >0)) \
                                            .reduceByKey(lambda c1, c2: c1 + c2)
mention_count.pprint()

In [13]:
ssc.start()

-------------------------------------------
Time: 2017-06-19 16:43:30
-------------------------------------------
('motogp', 2)
('26_danipedrosa', 3)
('nickyhayden', 1)
('lorenzo99', 2)
('marcmarquez93', 1)
('calcrutchlow', 1)
('senaldeportes', 1)
('dcabellor', 1)
('valeyellow46', 2)
('citytv', 1)
...

-------------------------------------------
Time: 2017-06-19 16:43:35
-------------------------------------------
('yonny68', 1)

-------------------------------------------
Time: 2017-06-19 16:43:40
-------------------------------------------
('christiandelbel', 1)
('nickyhayden', 1)
('valeyellow46', 1)



In [14]:
ssc.stop(False)

#### B) Calcular la frecuencia total acumulada de apariciones de cada hashtag en el campo body, actualizando un ranking con los 5 hashtags con mayor frecuencia de aparición.


<font color='red'>IMPORTANTE: En caso de fallo en tiempo de ejecución, reiniciar Kernel de jupyter(Kernal -> Restart) y la query funcionará </font>

In [4]:
# Context
time_interval = 5
rank_hast = 5

sc = spark.sparkContext
ssc = StreamingContext(sc, time_interval)

kafkaParams = {"metadata.broker.list": kafkaBrokerIPPort}
stream = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams)

def updateFunction(newValues, runningCount):
    if runningCount is None:
        runningCount = 0    
    return sum(newValues,runningCount)

def checkpoint_dir(check_directory):
    if os.path.exists(check_directory):
        shutil.rmtree(check_directory)    
    os.makedirs(check_directory)

In [5]:
check_directory = 'checkpoint'
checkpoint_dir(check_directory)
ssc.checkpoint(check_directory)

rows = stream.map(lambda line: line[1].split('\t'))
body = rows.flatMap(lambda row: row[6].split(' '))
hashtags= body.map(lambda word: (word, 1) if word.startswith('#') else (word, 0))
hashtags_count = hashtags.transform(lambda rdd: rdd.filter(lambda hash: hash[1] > 0)\
                                   .map(lambda item: (item[0], item[1])))\
                                    .updateStateByKey(updateFunction)                                           
            
top_rank = hashtags_count.transform(lambda rdd: rdd.sortBy(lambda tup: tup[1], ascending = False)
                                   .map(lambda ord_tup: list(ord_tup)).zipWithIndex()
                                   .filter(lambda ranked_tup: ranked_tup[1] < rank_hast)
                                   .map(lambda filt_tup: (filt_tup[0][0], filt_tup[0][1])))                                    
                                 

top_rank.pprint()

In [6]:
ssc.start()

-------------------------------------------
Time: 2017-06-19 16:44:30
-------------------------------------------
('#f1', 14)
('#gpmalasia', 3)
('#motogp', 2)
('#sepangcircuit', 1)

-------------------------------------------
Time: 2017-06-19 16:44:35
-------------------------------------------
('#f1', 34)
('#gpmalasia', 6)
('#motogp', 6)
('#defrenteconeldeporte', 3)
('#f1/maldonado', 3)

-------------------------------------------
Time: 2017-06-19 16:44:40
-------------------------------------------
('#f1', 35)
('#f1/maldonado', 11)
('#defrenteconeldeporte', 10)
('#gpmalasia', 6)
("#miiguel'dfultree", 6)



In [7]:
ssc.stop(False)

##### C) Calcular en una ventana temporal 20 segundos con offset de 10 segundos la frecuencia de aparición de cada uno de los 3 posibles tipos de tweets (TW-RT-MT).

In [8]:
# Context
time_interval = 5
time_window = 20
offset = 10

sc = spark.sparkContext
ssc = StreamingContext(sc, time_interval)

kafkaParams = {"metadata.broker.list": kafkaBrokerIPPort}
stream = KafkaUtils.createDirectStream(ssc, [topic], kafkaParams)

In [9]:
# ssc.checkpoint("checkpoint")

info_row = stream.map(lambda line: line[1].split('\t'))

tweet_types = info_row.map(lambda line: (line[9], 1))
frequency  =  tweet_types.reduceByKeyAndWindow(add, sub,
                                               windowDuration = time_window,
                                               slideDuration = offset)
frequency.pprint()

In [10]:
ssc.start()

-------------------------------------------
Time: 2017-06-19 16:45:00
-------------------------------------------
('MT', 14)
('TW', 33)
('RT', 7)

-------------------------------------------
Time: 2017-06-19 16:45:10
-------------------------------------------
('MT', 37)
('TW', 54)
('RT', 14)

-------------------------------------------
Time: 2017-06-19 16:45:20
-------------------------------------------
('MT', 57)
('TW', 21)
('RT', 23)



In [11]:
ssc.stop(False)